In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import shutil


In [ ]:
ROOT = Path('/data/datasets/dataset_project')
SEQS = []
SEQS = [*SEQS, 'hangpa00', 'hangpa01', 'hangpa02',
        'outdoor00', 'outdoor01', 'outdoor02', 'outdoor03']
SEQS = [*SEQS, 'itbt00', 'itbt01', 'itbt02', 'itbt03',
        'itbt_dark00', 'itbt_dark01', 'itbt_dark02', 'itbt_dark03']
SEQS.sort()
print(SEQS)


In [3]:
ROOT = Path('/data/datasets/dataset_project')
# SEQS = ['hangpa00', 'hangpa01', 'hangpa02',
#         'outdoor00', 'outdoor01', 'outdoor02', 'outdoor03']
SEQS = ['itbt00', 'itbt01', 'itbt02', 'itbt03',
        'itbt_dark00', 'itbt_dark01', 'itbt_dark02', 'itbt_dark03']

def getEnd2EndError(root: Path, seq: str):
  fast_lio2_fn = root / seq / 'pose_fast_lio2.txt'
  isam_fn = root / seq / 'pose_isam.txt'
  pose_fast_lio2 = np.genfromtxt(fast_lio2_fn, dtype=float)
  pose_isam = np.genfromtxt(isam_fn, dtype=float)
  # print(pose_fast_lio2.shape)
  # print(pose_isam.shape)

  # translations
  ts_fast = pose_fast_lio2[:, 1:4]
  ts_isam = pose_isam[:, 1:4]
  end2end_fast = np.linalg.norm(ts_fast[-1, :] - ts_fast[0, :])
  end2end_isam = np.linalg.norm(ts_isam[-1, :] - ts_isam[0, :])
  duration_fast = np.linalg.norm(pose_fast_lio2[-1,0] - pose_fast_lio2[0,0])
  duration_isam = np.linalg.norm(pose_isam[-1,0] - pose_isam[0,0])
  # print(f"end2end_fast: {end2end_fast}")
  # print(f"end2end_isam: {end2end_isam}")
  # print(f"duration_fast: {duration_fast}")
  # print(f"duration_isam: {duration_isam}")

  total_length1 = 0.0
  n_keyframe = ts_isam.shape[0]
  for i in range(n_keyframe-1):
    dist = np.linalg.norm(ts_isam[i,:] - ts_isam[i+1,:])
    total_length1 += dist
  # print(f"total_length1: {total_length1}")

  total_length2 = 0.0
  n_pose = ts_fast.shape[0]
  for i in range(n_pose-1):
    dist = np.linalg.norm(ts_fast[i,:] - ts_fast[i+1,:])
    total_length2 += dist
  # print(f"total_length2: {total_length2}")

  total_length = min(total_length1, total_length2)
  duration = min(duration_fast, duration_isam)

  e2e_fast_ratio = (end2end_fast / total_length) * 100.0
  e2e_isam_ratio = (end2end_isam / total_length) * 100.0
  print(f"{seq},{total_length:.2f}[m]/{duration:.2f}[sec],{end2end_fast:.3f} ({e2e_fast_ratio:.2f}%),{end2end_isam:.3f} ({e2e_isam_ratio:.2f}%)")
for seq in SEQS:
  getEnd2EndError(ROOT, seq)


itbt00,134.47[m]/167.40[sec],0.641 (0.48%),0.113 (0.08%)
itbt01,139.23[m]/181.60[sec],0.134 (0.10%),0.069 (0.05%)
itbt02,129.33[m]/189.70[sec],0.075 (0.06%),0.123 (0.10%)
itbt03,412.09[m]/505.40[sec],0.666 (0.16%),0.073 (0.02%)
itbt_dark00,135.26[m]/146.60[sec],0.139 (0.10%),0.091 (0.07%)
itbt_dark01,132.65[m]/159.50[sec],0.656 (0.49%),0.180 (0.14%)
itbt_dark02,131.26[m]/179.80[sec],0.051 (0.04%),0.080 (0.06%)
itbt_dark03,381.12[m]/425.10[sec],0.194 (0.05%),0.151 (0.04%)
